In [72]:
# imports
import drgmum
from drgmum.toolkit.dataset import SmartDataset
from drgmum.chemdata.fingerprints import molprint2d_count_fingerprinter
from drgmum.chemdata.chembl.sqlite_queries import get_smiles
from drgmum.chemdata.utils.pandas_utils import two_level_series_to_csr
import logging
logging.basicConfig()
import os
data_folder = '/var/data/users/local/pocha/data'
chembl21db = os.path.join(data_folder, 'DRGMUM/chembl_21.db')
chembl_ids_file = os.path.join(data_folder, 'SCFP/Random_compounds_100.sdf')
similarity_matrix_file = os.path.join(data_folder, 'SCFP/Similarity150Dawid.csv')
folder_with_pairs = os.path.join(data_folder, 'SCFP/pairs')
from load_constraints import get_chembls, load_similarity_matrices
from sklearn.base import BaseEstimator, ClusterMixin
from sklearn.cluster import KMeans
from sklearn.utils.estimator_checks import check_estimator
from sklearn.model_selection import train_test_split
random_state = 666

In [68]:
# load data

# 1. load chembl IDs
chembl_ids = get_chembls(chembl_ids_file)
smiles = get_smiles(chembl21db, chembl_ids)

# 2. load fingerprints
a = molprint2d_count_fingerprinter(smiles) # nieważne
sprase_fp, row_labels, col_labels = two_level_series_to_csr(a)

# 3. load constraints
bin_sim, scale_sim, mapping_idx_chembl = load_similarity_matrices(similarity_matrix_file, chembl_ids_file, folder_with_pairs)
bin_sim_list = zip(zip(*bin_sim.nonzero()), bin_sim.data)
scale_sim_list = zip(zip(*scale_sim.nonzero()), scale_sim.data)

INFO:drgmum.chemdata.chembl.sqlite_queries:Extracting SMILES from the database...
INFO:drgmum.chemdata.chembl.sqlite_queries:Done
INFO:drgmum.chemdata.fingerprints._openbabel:Processing 100 SMILES...
100it [00:00, 199159.73it/s]
645it [00:00, 316931.36it/s]
100%|██████████| 2144/2144 [00:00<00:00, 182531.32it/s]

21 pairs were omitted


In [73]:
# podział więzów na foldy
train_test_split(bin_sim_list, train_size=0.8, random_state=random_state, )

[[((39, 40), -1),
  ((84, 93), -1),
  ((57, 11), -1),
  ((31, 78), -1),
  ((43, 84), 1),
  ((97, 93), -1),
  ((4, 51), -1),
  ((61, 1), 1),
  ((1, 29), 1),
  ((44, 6), -1),
  ((6, 58), -1),
  ((62, 87), 1),
  ((52, 30), -1),
  ((6, 19), -1),
  ((43, 58), -1),
  ((85, 85), 1),
  ((74, 94), -1),
  ((6, 15), -1),
  ((25, 17), -1),
  ((31, 31), 1),
  ((24, 62), 1),
  ((60, 72), -1),
  ((62, 66), -1),
  ((2, 1), 1),
  ((68, 75), -1),
  ((8, 89), -1),
  ((15, 22), 1),
  ((95, 30), -1),
  ((2, 54), 1),
  ((19, 90), -1),
  ((47, 52), -1),
  ((41, 52), -1),
  ((99, 2), 1),
  ((20, 32), -1),
  ((84, 83), -1),
  ((58, 49), -1),
  ((30, 80), -1),
  ((87, 39), -1),
  ((86, 73), -1),
  ((71, 75), -1),
  ((16, 80), -1),
  ((93, 93), 1),
  ((80, 37), -1),
  ((91, 91), 1),
  ((40, 52), -1),
  ((73, 73), 1),
  ((10, 29), -1),
  ((30, 95), -1),
  ((2, 99), 1),
  ((32, 33), -1),
  ((54, 35), -1),
  ((79, 10), -1),
  ((5, 73), -1),
  ((60, 60), 1),
  ((55, 26), -1),
  ((7, 99), -1),
  ((99, 37), -1),
  ((5

In [ ]:
# define score

In [34]:
# define model
class KMeans_SCFP(KMeans):
    def score():
        pass
    
check_estimator(KMeans_SCFP)

TypeError: score() takes no arguments (3 given)

In [ ]:
# define models and hiperparameters

In [ ]:
# define experiment